In [38]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "user"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, "nv-desktop-services.apporto.com", 30813, "AAC", "animals")

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('www.snhu.edu Alex Crosswhite Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        #Radio Items to select the rescue filter options
        className='row',
        style={'display': 'flex'},
        children=[
                html.Button(id='button-one',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='button-two',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='button-three',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='button-four', n_clicks=0, children='reset')          
        ]
    ),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         editable=True,
                         style_cell={'textAlign': 'left'},
                         style_cell_conditional=[{'if': {'column_id': s}, 
                                                  'textAlign': 'left'} for s in ['animal_type', 
                                                                                 'color', 
                                                                                 'breed', 
                                                                                 'animal_id']],
                         style_data_conditional=[{'if': {'row_index': 'odd'}, 
                                                  'backgroundColor': 'rgb(200, 200, 200)'}],
                         style_header={'backgroundColor': 'rgb(230, 0, 0)', 
                                       'fontWeight': 'bold'},
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10,

                        ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data')],
              [Input('button-one', 'n_clicks'),Input('button-two','n_clicks'), 
               Input('button-three','n_clicks'),Input('button-four','n_clicks')]
             )
def update_dashboard(button1, button2, button3, button4):
    # Add code to filter the interactive data table with MongoDB queries
    if (int(button1) >= 1):
        quiry_criteria = {'$and': [ 
            {'$or': [ 
                {'breed':'Labrador Retriever Mix'}, 
                {'breed':'Chesapeake Bay Retriever'},
                {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, 
            {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}
        
        df = pd.DataFrame.from_records(AnimalShelter.read(quiry_criteria))
        button2, button3, bt4 = 0

    elif (int(button2)>= 1):
        quiry_criteria = {'$and': [ 
            {'$or': [ 
                {'breed':'German Shepherd'}, 
                {'breed':'Alaskan Malamute'},
                {'breed':'Old English Sheepdog'},
                {'breed':'Siberian Husky'},
                {'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, 
            {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}
        
        df = pd.DataFrame.from_records(AnimalShelter.read(quiry_criteria))
        button1, button3 ,bt4 = 0

    elif (int(button3)>=1):
        quiry_criteria = {'$and': [ 
            {'$or': [ 
                {'breed':'Doberman Pinscher'}, 
                {'breed':'German Sheperd'},
                {'breed':'Golden Retriever'},
                {'breed':'Bloodhound'},
                {'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, 
            {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}
        
        df = pd.DataFrame.from_records(AnimalShelter.read(quiry_criteria))
        bt1, button2, bt4 = 0

    elif(int(button4)>=1):
        df = pd.DataFrame.from_records(AnimalShelter.read())
        bt1, button2, button3 = 0

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')


    return data


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)

def update_graphs(viewData):
    ###FIX ME ####
    dff = pd.DataFrame.from_dict(viewData)
    new_df = dff.breed.value_counts()
    new_df.name = "breed"
    new_df = new_df.to_frame()
    new_df.reset_index(inplace=True)
    
    # return updated graph data
    return [dcc.Graph(figure = px.pie(new_df, values='breed', names="index"))]
# Display the breeds of animal based on quantity represented in
# the data table

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

def update_map(viewData, index):
    #FIXME: Add in the code for your geolocation chart
    #If you completed the Module Six Assignment, you can copy in the code you created here.
    if index is None or not viewData:
        return [dl.Map(style={'width': '1000px', 'height':'500px'}, center=[30.75,-97.48], zoom=10)]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] 
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),
                          dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row,9])])
                      ])
        ])
    ]
    
def display_map(viewData, index):
    return update_map(viewData, index)



app.run_server(debug=True)


Dash app running on http://127.0.0.1:10789/
